## Analysis scripts for investigation of EKE in moist QG simulations

Generates Figures 1-4 in Lutsko et al (2024) "Atmospheric Moisture Decreases Mid-Latitude Eddy Kinetic Energy". The moist QG code is available from https://github.com/nicklutsko/moist_QG_channel

## Part 1: Homogeneous model

Note that the simulations directly output EKE and growth rates, which were copied into the arrays. This makes Figure 1

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pylab as plt
import scipy.signal as ss
import scipy.stats as st

In [ ]:
#Vary L:
L = [0., 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
gr = [0.12, 0.15, 0.17, 0.21, 0.22, 0.25, 0.27, 0.29, 0.31, 0.33, 0.35, 0.36]
eke = [2155.627136, 2307.874517, 2798.559454, 3482.941081, 4313.828404, 4730.450517, 5411.312516, 5997.891174, 6648.926213, 7159.865929]

#Vary C:
C = [0., 0.5, 1., 1.39, 2., 2.5, 3., 3.5, 4., 4.5, 5.]
eke2 = [2178.384465, 2286.777367, 2352.571572, 2668.866263, 2798.559454, 3089.783547, 3182.047681, 3347.763173, 3483.343521, 3632.888642, 3770.284162]

#Extra simulations with E set to 0.4, then varying L
gr_e04 = [0.12, 0.16, 0.18, 0.2, 0.22, 0.247, 0.2756, 0.298, 0.3196, 0.341, 0.353, 0.355]
eke_e04 = [2155.627136, 2099.4945, 2354.61018135307, 2594.954722806772, 2966.169528866504, 3342.6683167387496, 3699.9461]


In [ ]:
fig = plt.figure( figsize = (10, 7) )
fig.subplots_adjust(left =0.1, bottom = 0.1, top = 0.93, hspace = 0.4, wspace = 0.35, right = 0.97)

ax = plt.subplot(2, 2, 1)
plt.title("a) linear growth rates", loc = "left", fontsize = 18)

plt.plot(L, gr, 'kx-', linewidth = 0.75, markersize = 8)

plt.ylim([0.1, 0.4])
plt.xlim([-0.05, 1.05])

plt.yticks([0.1, 0.2, 0.3, 0.4], fontsize = 16)
plt.xticks(fontsize = 16)

#plt.xlabel("L", fontsize = 16)
plt.ylabel("growth rate", fontsize = 18.)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.axvline( x = 0.45, color = 'k', linestyle = ":", linewidth = 0.5)

ax = plt.subplot(2, 2, 2)
plt.title("b)", loc = "left", fontsize = 18)

#Linear regression for scaling:
stab = (1. - np.array(L)) / (1. + 2. * np.array(L))
test = (-(stab + 1.) + np.sqrt((stab + 1) ** 2 + 4. * stab)) / (2. * stab)
slope, inter = st.linregress(test, gr)[:2]

plt.plot(stab, gr, 'kx')
plt.plot(stab, slope * test + inter, 'r')
plt.ylim([0.1, 0.4])
plt.xlim([1.05, -0.05])

plt.yticks([0.1, 0.2, 0.3, 0.4], fontsize = 16)
plt.xticks(fontsize = 16)

plt.legend(["simulation", "ZG05"], fontsize = 14, loc = "upper left")


#plt.xlabel("effective stability", fontsize = 16)
#plt.ylabel("growth rate", fontsize = 16.)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax = plt.subplot(2, 2, 3)
plt.title("c) nonlinear simulations", loc = "left", fontsize = 18)

plt.plot(L[:10], eke, 'ko-', linewidth = 0.75, markersize = 6)

plt.xlim([-0.05, 1.05])
plt.ylim([1500., 8000.])

plt.yticks([2000., 4000., 6000., 8000.], fontsize = 16)
plt.xticks(fontsize = 16)

plt.xlabel("L", fontsize = 18)
plt.ylabel("EKE", fontsize = 18.)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.axvline( x = 0.45, color = 'k', linestyle = ":", linewidth = 0.5)


ax = plt.subplot(2, 2, 4)
plt.title("d)", loc = "left", fontsize = 18)

#Linear regression for scaling:
stab = (1. - np.array(L)) / (1. + 2. * np.array(L))
stab2 = (1. - 0.2) / (1. + np.array(C) * 0.2)
plt.plot(stab[:10], eke, 'ko')
plt.plot(stab2, eke2, 'kv')
nstab = np.concatenate((stab[:10], stab2))
s = np.argsort(nstab)
neke = np.concatenate((eke, eke3))[s]
nstab = np.sort(nstab)

test = ((-(nstab + 1.) + np.sqrt((nstab + 1) ** 2 + 4. * nstab)) / (2. * nstab)) ** 2
slope, inter = st.linregress(test, neke)[:2]
plt.plot(nstab, slope * test + inter, 'r')


plt.xlim([1.05, -.05])
plt.ylim([1500., 8000.])

plt.yticks([2000., 4000., 6000., 8000.], fontsize = 16)
plt.xticks(fontsize = 16)

plt.xlabel("effective stability", fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.legend(["vary $L$", "vary $C$", "ZG05"], loc = "upper left", fontsize = 14)

plt.savefig("images/homogeneous_QG_EKE.pdf")
plt.savefig("images/homogeneous_QG_EKE.png")

## Part 2: Channel model

In [ ]:
N = 128 #zonal size of spectral decomposition
N2 = 128 #meridional size of spectral decomposition
Lx = 72. #size of x -- stick to multiples of 10
Ly = 96. #size of y -- stick to multiples of 10

beta = 0.2

L = ["0", "0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9"]
Lf = [0., 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

l = len( L ) 

x = np.linspace( -Lx / 2, Lx / 2, N ) 
y = np.linspace( -Ly / 2, Ly / 2, N2 ) 

#Wavenumbers:
kk = np.fft.rfftfreq( N, Lx / float(N) / 2. / np.pi ) #zonal wavenumbers
ll = np.fft.fftfreq( N2, Ly / float(N2) / 2. / np.pi ) #meridional wavenumbers

In [ ]:
#First load data
mean_zu1 = np.zeros( ( l, N2)) #upper layer winds
mean_zu2 = np.zeros( ( l, N2)) #lower layer winds
mean_ztau = np.zeros( ( l, N2)) #temperature
mean_zpre = np.zeros( ( l, N2)) #precip

mean_zeke = np.zeros( ( l, N2)) #upper layer eke
mean_zeke2 = np.zeros( ( l, N2)) #lower layer eke
mean_zehf = np.zeros( ( l, N2)) #eddy heat flux

mean_eape = np.zeros( ( l, N2)) #eddy APE
mean_eape_eke = np.zeros( ( l, N2)) #conversion of EAPE to EKE
mean_Peddy = np.zeros( ( l, N2)) #eddy precipitation

for i in range( l ):
    print("Doing: ", i)
    
    ds = xr.open_dataset( "/data/model_output/moist_QG_channel-main_v5/run_L_" + L[i] + "/N" + str(N2) + "_" + L[i] + "_2.0_0.1_1.0.nc")
    
    mean_zu1[i] = np.mean( ds.zu1[:-25], axis = 0)
    mean_zu2[i] = np.mean( ds.zu2[:-25], axis = 0)
    mean_ztau[i] = np.mean( ds.ztau[:-25], axis = 0)
    mean_zpre[i] = np.mean( ds.zP[:-25], axis = 0)

    mean_zeke[i] = np.mean( ds.zeke1[:-25], axis = 0)
    mean_zeke2[i] = np.mean( ds.zeke2[:-25], axis = 0)
    mean_zehf[i] = np.mean( ds.zehf1[:-25], axis = 0)

    mean_eape[i] = np.mean( ds.eape[:-25], axis = 0)
    mean_eape_eke[i] = np.mean( ds.eape_eke[:-25], axis = 0)
    mean_Peddy[i] = np.mean( ds.Peddy[:-25], axis = 0)

In [ ]:
#Make radiative equilibrium slope, need this to calculate contribution of radiation to energy budget
U_1 = 1.
sigma = 3.5

u_eq = np.zeros( ( N2, N ) )

for i in range( N2 ):
    y1 = float( i - N2 /2) * (y[1] - y[0] )
    y2 = float(min(i, N2 -i - 1)) * (y[1] - y[0] )
    u_eq[i, :] = U_1 * ( 1. / (np.cosh(abs(y1/sigma)))**2 - 1. / (np.cosh(abs(y2/sigma)))**2  )

psi_Rc = -np.fft.rfft2(  u_eq ) / 1.j / ll[:, np.newaxis]
psi_Rc[0] = 0.
psi_R = np.fft.irfft2( psi_Rc )

In [ ]:
#Now calculate EKE, etc
EKE = np.zeros( l )
APE = np.zeros( l )
EAPE = np.zeros( l )

#conversion terms
tau_P = np.zeros( l )
CAPE = np.zeros( l ) #conversion to APE
rad = np.zeros(l)
EAPE_eke = np.zeros( l )
P_eddy = np.zeros( l )

#Calculate lower layer PV gradient
PV2 = beta - (mean_zu1 - mean_zu2) - np.gradient(np.gradient(mean_zu1, y, axis = 1), axis = 1)

mean_eke = mean_zeke[:, :] + mean_zeke2[:, :]

for i in range( l ):
    #find where PV gradient reverses
    ls = np.where(PV2[i] <= 0.)
    temp = np.mean(mean_ztau[i, ls] ** 2)  - np.mean(mean_ztau[i, ls]) ** 2
    
    EKE[i] = np.mean(mean_eke[i, ls])
    APE[i] = np.mean(temp) / 2.
    EAPE[i] = np.mean(mean_eape[i, ls] )

    #now conversion terms
    tau_P[i] = Lf[i] * np.mean(mean_ztau[i, ls] *  mean_zpre[i, ls])
    grad = np.gradient(mean_zehf[i], y)
    CAPE[i] = -np.mean( mean_ztau[i, ls] * grad[ls])
    rad[i] = np.mean( mean_ztau[i, ls]* (psi_R[ls, 0] - mean_ztau[i, ls])) / 100.
    EAPE_eke[i] = np.mean(mean_eape_eke[i, ls] )
    if i > 0:
        P_eddy[i] = Lf[i] * np.mean(mean_Peddy[i, ls])

Finally plot Figures 2 and 3:

In [ ]:
fig = plt.figure( figsize = (16, 4) )
fig.subplots_adjust(left =0.08, bottom = 0.15, top = 0.9, hspace = 0.4, wspace = 0.4, right = 0.95)

ax = plt.subplot(1, 3, 1)
plt.title("a)", fontsize = 20, loc = "left")
plt.plot(Lf[:], EKE[:], 'kx-')

plt.xlim([-0.03, 0.95])
plt.ylim([0., 0.5])

plt.yticks([0., 0.1, 0.2, 0.3, 0.4, 0.5], fontsize = 18)
plt.xticks(fontsize = 18)

plt.xlabel("L", fontsize = 20)
plt.ylabel("EKE", fontsize = 20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.axvline( x = 0.45, color = 'k', linestyle = ":", linewidth = 0.5)

plt.text(0.2, 0.05, "weak", fontsize = 18)
plt.text(0.53, 0.05, "strong", fontsize = 18)

ax = plt.subplot(1, 3, 2)
plt.title("b)", fontsize = 20, loc = "left")

plt.plot(Lf[:], APE[:], 'kx-')
plt.xlim([-0.03, 0.95])
plt.ylim([0., 2.2])

plt.yticks([0., 0.5, 1., 1.5, 2.], fontsize = 18)
plt.xticks(fontsize = 18)

plt.xlabel("L", fontsize = 20)
plt.ylabel("MAPE", fontsize = 20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.axvline( x = 0.45, color = 'k', linestyle = ":", linewidth = 0.5)

plt.text(0.2, 0.05 * 2.2/0.5, "weak", fontsize = 18)
plt.text(0.53, 0.05* 2.2/0.5, "strong", fontsize = 18)


ax = plt.subplot(1, 3, 3)
plt.title("c)", fontsize = 20, loc = "left")

plt.plot(APE[:], EKE[:], 'kx-')

plt.xlim([0., 2.2])
plt.ylim([0., 0.5])

plt.yticks([0., 0.1, 0.2, 0.3, 0.4, 0.5], fontsize = 18)
plt.xticks(fontsize = 18)

plt.xlabel("MAPE", fontsize = 20)
plt.ylabel("EKE", fontsize = 20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.axvline( x = 1.5, color = 'k', linestyle = ":", linewidth = 0.5)

plt.text(0.8, 0.05, "strong", fontsize = 18)
plt.text(1.7, 0.05, "weak", fontsize = 18)

plt.savefig("images/2layer_QG_EKE.pdf")
plt.savefig("images/2layer_QG_EKE.png")

In [ ]:
fig = plt.figure( figsize = (10, 4) )
fig.subplots_adjust(left =0.13, bottom = 0.15, top = 0.9, hspace = 0.4, wspace = 0.3, right = 0.97)

ax = plt.subplot(1, 2, 1)
plt.title("a) MAPE budget", fontsize = 18, loc = "left")

plt.plot(Lf[:], CAPE[:], 'kD-', markersize = 9)
plt.plot(Lf[:], -rad[:]-tau_P[:] - CAPE[:], 'ko-', markersize = 9)
plt.plot(Lf[:], tau_P[:], 'kx:', markersize = 9)
plt.plot(Lf[:], rad[:], 'kv-', markersize = 9)

plt.legend(["-conv(EAPE)", "conv(ZKE)", "precip", "rad"], ncol = 2, fontsize = 14, loc = "upper left")

plt.axhline(y = 0, color = 'k')

plt.yticks([-0.02, 0., 0.02, 0.04], fontsize = 16)
plt.xticks(fontsize = 16)

plt.xlabel("L", fontsize = 18)
plt.ylabel("tendency", fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xlim([-0.03, 0.95])
plt.ylim([-0.025, 0.05])

ax = plt.subplot(1, 2, 2)
plt.title("b) EAPE budget", fontsize = 18., loc = "left")

plt.plot(Lf, -CAPE, 'kD-', markersize = 9)
plt.plot(Lf, EAPE_eke, 'ko-', markersize = 9)
plt.plot(Lf, P_eddy, 'kx:', markersize = 9)
plt.plot(Lf, -EAPE / 100., 'kv-', markersize = 9)
#plt.plot(Lf, CAPE - EAPE_eke + EAPE / 100., 'kx:') #check residual is close to P_eddy

plt.legend(["conv(EAPE)", "-conv(EKE)", "precip", "rad"], ncol = 2, loc = "upper right", fontsize = 14)

plt.axhline(y = 0, color = 'k')


plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)

plt.xlabel("L", fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlim([-0.03, 0.95])
plt.ylim([-0.025, 0.025])

plt.savefig("images/2layer_QG_budget.pdf")
plt.savefig("images/2layer_QG_budget.png")

Finally plot zonal-mean precipitation and temperature (Figure 4)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

gamma= 1.
rels=  { 
  "pair2_a": [0.0117647061124444, 0.05098039284348488, 0.6352941393852234], 
  "pair2_b": [0.07058823853731155, 0.658823549747467, 0.07058823853731155], 
  "aug2": [0.9058823585510254, 0.5411764979362488, 0.03529411926865578], 
  "aug1_add": [0.0, 0.40392157435417175, 0.1882352977991104], 
  "aug2_add": [0.6666666865348816, 0.1411764770746231, 0.23137255012989044], 
  "grey": [0.4431372582912445, 0.4470588266849518, 0.4470588266849518], 
  "lead3": [0.07058823853731155, 0.658823549747467, 0.07058823853731155], 
  "lead2": [0.95686274766922, 0.1764705926179886, 0.12156862765550613], 
  "lead1": [0.07450980693101883, 0.3294117748737335, 0.6509804129600525], 
  "gridcolor": [0.8196078538894653, 0.8196078538894653, 0.8196078538894653], 
  "axiscolor": [0.11372549086809158, 0.11372549086809158, 0.11372549086809158], 
  "cascade3": [0.4470588266849518, 0.6196078658103943, 0.843137264251709], 
  "cascade2": [0.07450980693101883, 0.3294117748737335, 0.6509804129600525], 
  "plus": [0.95686274766922, 0.1764705926179886, 0.12156862765550613], 
  "aug1": [0.07058823853731155, 0.658823549747467, 0.07058823853731155], 
  "cascade1": [0.0117647061124444, 0.05098039284348488, 0.6352941393852234],
   "minus": [0.07450980693101883, 0.3294117748737335, 0.6509804129600525], 
   "cascade4": [0.7333333492279053, 0.7921568751335144, 0.8823529481887817], 
    "green": [0.05882352963089943, 0.545098066329956, 0.05882352963089943] ,
   "lightgreen": [144. /255., 238./255., 144./255.],
   "black": [0.11372549086809158, 0.11372549086809158, 0.11372549086809158]}

cascade_r=LinearSegmentedColormap.from_list("my_colormap",
                                (rels['grey'], rels['cascade4'], rels['cascade3'], rels['cascade2'], rels['cascade1'], rels['green'] ), N=l, gamma=gamma)

evenly_spaced_interval = np.linspace(0., 1., l)
colors = [cascade_r(x) for x in evenly_spaced_interval]
colors2 = [plt.cm.Reds(x) for x in evenly_spaced_interval]

centers = np.zeros( l )

def make_axis( ax ):
    
    plt.xlim([-25., 35.])
    #plt.xlabel("y", fontsize = 10)
    
    plt.xticks([-20., -10, 0., 10., 20., 30.], fontsize = 16)
    plt.yticks(fontsize = 16)
    plt.rc('axes.spines', top= False, right=False )
    return 0

In [ ]:
fig = plt.figure( figsize = (10, 4) )
fig.subplots_adjust(left =0.1, bottom = 0.15, top = 0.9, hspace = 0.4, wspace = 0.2, right = 0.97)

ax = plt.subplot(1, 2, 1)
plt.title("a) Precipitation", loc = "left", fontsize = 18)

for i in range( l):
    plt.plot( y, mean_zpre[i] , color = colors[i] )

plt.ylim([0., 0.24])
plt.yticks([0., 0.05, 0.1, 0.15, 0.2], fontsize = 16)

plt.legend(["Dry", "L = 0.1", "L = 0.2", "L = 0.3", "L = 0.4", "L = 0.5", "L = 0.6", "L = 0.7", "L = 0.8", "L = 0.9"], 
           frameon = False, loc = "upper right", ncol = 2, fontsize = 14)

make_axis( ax )

plt.xlabel("y", fontsize = 18)
plt.axvline(x = 0, color = 'k', linewidth = 0.5)

ax = plt.subplot(1, 2, 2)
plt.title("b) Temperature", loc = "left", fontsize = 18)

for i in range( l):
    plt.plot( y, mean_ztau[i], color = colors[i] )

make_axis( ax )
plt.yticks([-4., -2., 0., 2., 4.], fontsize = 16)

plt.xlabel("y", fontsize = 18)
plt.axvline(x = 0, color = 'k', linewidth = 0.5)
plt.axhline(y = 0, color = 'k', linewidth = 0.5)

plt.savefig("images/2layer_QG_PT.pdf")
plt.savefig("images/2layer_QG_PT.png")